# <div align='center'> 分类任务测试 </div>

In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -v -p numpy,sklearn,pandas
%watermark -v -p cv2,PIL,matplotlib
%watermark -v -p torch,torchvision,torchaudio
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, Javascript
display(HTML('<style>.container { width:%d%% !important; }</style>' % 80))

def _IMPORT_(x):
    try:
        exec(x, globals())
    except:
        pass


CPython 3.6.9
IPython 7.16.1

numpy 1.19.5
sklearn 0.0
pandas 1.1.5
CPython 3.6.9
IPython 7.16.1

cv2 4.5.3
PIL 8.3.1
matplotlib 3.3.4
CPython 3.6.9
IPython 7.16.1

torch 1.8.1+cu101
torchvision 0.9.1+cu101
torchaudio not installed


In [2]:
###
### Common ###
###

import sys, os, io, time, random, math
import json, base64, requests
import os.path as osp

_IMPORT_('import numpy as np')
_IMPORT_('import pandas as pd')
_IMPORT_('from tqdm.notebook import tqdm')

def print_progress_bar(x):
    print('\r', end='')
    print('Progress: {}%:'.format(x), '%s%s' % ('▋'*(x//2), '.'*((100-x)//2)), end='')
    sys.stdout.flush()

def img2b64(x):
    if isinstance(x, bytes):
        return base64.b64encode(x).decode()
    elif isinstance(x, str):
        if x.startswith('http'):
            return base64.b64encode(requests.get(x).content).decode()
        with open(x, 'rb') as fr:
            return base64.b64encode(fr.read()).decode()
    raise

###
### Display ###
###

_IMPORT_('import cv2')
_IMPORT_('from PIL import Image')
_IMPORT_('from torchvision.utils import make_grid')
_IMPORT_('import matplotlib.pyplot as plt')
_IMPORT_('import plotly')
_IMPORT_('import plotly.graph_objects as go')
_IMPORT_('import ipywidgets as widgets')
_IMPORT_('from ipywidgets import interact, interactive, fixed, interact_manual')

# plotly.offline.init_notebook_mode(connected=False)

plt.rcParams['figure.figsize'] = (12.0, 8.0)

def show_table(headers, data, width=900):
    ncols = len(headers)
    width = int(width / ncols)
    lralign = []
    caption = []
    for item in headers:
        astr = ''
        if item[0] == ':':
            astr = ':'
            item = item[1:]
        astr += '---'
        if item[-1] == ':':
            astr += ':'
            item = item[:-1]
        lralign.append(astr)
        caption.append(item)
    captionstr = '|'.join(caption) + chr(10)
    lralignstr = '|'.join(lralign) + chr(10)
    imgholdstr = '|'.join(['<img width=%d/>' % width] * ncols) + chr(10)
    table = captionstr + lralignstr + imgholdstr
    is_dict = isinstance(data[0], dict)
    for row in data:
        if is_dict:
            table += '|'.join([f'{row[c]}' for c in caption]) + chr(10)
        else:
            table += '|'.join([f'{col}' for col in row]) + chr(10)
    return Markdown(table)

def show_video(vidsrc, width=None, height=None):
    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if vidsrc.startswith('http'):
        data_url = vidsrc
    else:
        mp4 = open(vidsrc, 'rb').read()
        data_url = 'data:video/mp4;base64,' + base64.b64encode(mp4).decode()
    return HTML('<video %s %s controls src="%s" type="video/mp4"/>' % (W, H, data_url))

def show_image(imgsrc, width=None, height=None):
    if isinstance(imgsrc, np.ndarray):
        img = imgsrc
        if width or height:
            if width and height:
                size = (width, height)
            else:
                rate = img.shape[1] / img.shape[0]
                if width:
                    size = (width, int(width/rate))
                else:
                    size = (int(height*rate), height)
            img = cv2.resize(img, size)
            plt.figure(figsize=(3*int(size[0]/80+1), 3*int(size[1]/80+1)), dpi=80)
        plt.axis('off')
        if len(img.shape) > 2:
            plt.imshow(img);
        else:
            plt.imshow(img, cmap='gray');
        return

    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if imgsrc.startswith('http'):
        data_url = imgsrc
    else:
        if len(imgsrc) > 512:
            data_url = 'data:image/jpg;base64,' + imgsrc
        else:
            img = open(imgsrc, 'rb').read()
            data_url = 'data:image/jpg;base64,' + base64.b64encode(img).decode()
    return HTML('<img %s %s src="%s"/>' % (W, H, data_url))

    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if image_path.startswith('http'):
        data_url = image_path
    else:
        img = open(image_path, 'rb').read()
        data_url = 'data:image/jpg;base64,' + base64.b64encode(img).decode()
    return HTML('<img %s %s src="%s"/>' % (W, H, data_url))

def im_read(url, rgb=True, size=None):
    if url.startswith('http'):
        response = requests.get(url)
        if response:
            imgmat = np.frombuffer(response.content, dtype=np.uint8)
            img = cv2.imdecode(imgmat, cv2.IMREAD_COLOR)
        else:
            return None
    else:
        img = cv2.imread(url)
        
    if rgb:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if size:
        if isinstance(size, int):
            size = (size, size)
        img = cv2.resize(img, size, interpolation=cv2.INTER_AREA)
    return img


In [2]:
import os
os.environ.setdefault('HOST_IP', '172.21.0.16')
os.environ.setdefault('NET_IP', '81.70.134.138')
from k12libs.utils.nb_easy import k12ai_run_project

In [3]:
backbones = [
    'resnet18',           # 0
    'resnet50',           # 1
    'densenet121',        # 2
    'mobilenet_v2',       # 3
    'squeezenet1_0',      # 4
    'squeezenet1_1',      # 5
    'shufflenet_v2_x0_5', # 6
    'shufflenet_v2_x1_0', # 7
]

batch_sizes = [8, 16, 32, 64]
input_sizes = [28, 32, 64, 128, 256]

## 任务

In [4]:
project = k12ai_run_project(
    debug=True, # tb_port=9002,
    framework='k12cv', task='cls',
    network=backbones[1], dataset='rmnist')

Box(children=(Box(children=(VBox(children=(Tab(children=(VBox(children=(HBox(children=(Text(value='cls', conti…

## 调用API

### Train

In [7]:
op = 'train.start'
appId = 'k12ai.popkey'
user = 'user01'
uuid = 'uuid01'
dataset_name = 'mnist01'
host = '116.85.5.40'
port = 8119
uri = 'k12ai/framework/execute'
popapi = f'http://{host}:{port}/k12ai/private/popmsg'
params = project.get_all_kv()
params['data.data_dir'] = f'oss://data-platform/datasets/{user}/{uuid}/{dataset_name}'
params['dataset'] = 'listdir'
params['data.dataset'] = dataset_name
params['solver.max_epoch'] = 160
params['solver.display_iter'] = 10
params['solver.test_interval'] = 20
params = json.dumps(params, indent=4)
print(params)

{
    "task": "cls",
    "method": "image_classifier",
    "data.include_val": false,
    "dataset": "listdir",
    "data.workers": 1,
    "data.drop_last": false,
    "data.image_tool": "cv2",
    "data.input_mode": "BGR",
    "solver.display_iter": 10,
    "solver.test_interval": 20,
    "solver.save_iters": 1024,
    "train.batch_size": 32,
    "val.batch_size": 32,
    "test.batch_size": 32,
    "train.data_transformer.fit_stride": 1,
    "train.data_transformer.size_mode": "fix_size",
    "train.data_transformer.input_size": [
        28,
        28
    ],
    "train.data_transformer.align_method": "scale_and_pad",
    "train.data_transformer.pad_mode": "pad_border",
    "_k12.train.random_border.bool": false,
    "_k12.train.random_brightness.bool": false,
    "_k12.train.random_contrast.bool": false,
    "_k12.train.random_crop.bool": false,
    "_k12.train.random_det_crop.bool": false,
    "_k12.train.random_gauss_blur.bool": false,
    "_k12.train.random_hsv.bool": false,
    

In [9]:
data = json.loads('''{
    "op":"%s",
    "token": "123456",
    "appId": "%s",
    "user": "%s",
    "service_name": "k12cv",
    "service_uuid": "%s",
    "service_params": %s
}''' % (op, appId, user, uuid, params))

api = 'http://%s:%d/%s' % (host, port, uri)
print("Input:\n")
print(data)
json.loads(requests.post(url=api, json=data).text)
def show_progress(popapi, msgkey, waitcnt=10):
    cnt = 1
    while True:
        result = json.loads(requests.get(url=f'{popapi}?key={msgkey}').text)
        for r in result:
            r = json.loads(r)
            for d in r['data']:
                if d['type'] == 'text':
                    p = d['data']
                    if 'progress' in p['title']:
                        v = int(float(p['payload']))
                        print_progress_bar(v)
                        if v >= 100:
                            return
            cnt = 0
        cnt += 1
        time.sleep(1)
        if cnt > waitcnt:
            print('timeout')
            break
print("\n\n")
show_progress(popapi, appId + '.metrics', 20)

Input:

{'op': 'train.start', 'token': '123456', 'appId': 'k12ai.popkey', 'user': 'user01', 'service_name': 'k12cv', 'service_uuid': 'uuid01', 'service_params': {'task': 'cls', 'method': 'image_classifier', 'data.include_val': False, 'dataset': 'listdir', 'data.workers': 1, 'data.drop_last': False, 'data.image_tool': 'cv2', 'data.input_mode': 'BGR', 'solver.display_iter': 10, 'solver.test_interval': 20, 'solver.save_iters': 1024, 'train.batch_size': 32, 'val.batch_size': 32, 'test.batch_size': 32, 'train.data_transformer.fit_stride': 1, 'train.data_transformer.size_mode': 'fix_size', 'train.data_transformer.input_size': [28, 28], 'train.data_transformer.align_method': 'scale_and_pad', 'train.data_transformer.pad_mode': 'pad_border', '_k12.train.random_border.bool': False, '_k12.train.random_brightness.bool': False, '_k12.train.random_contrast.bool': False, '_k12.train.random_crop.bool': False, '_k12.train.random_det_crop.bool': False, '_k12.train.random_gauss_blur.bool': False, '_k12.t

[metrics doc](http://116.85.5.40:8118/notebooks/docs/metrics_types.ipynb)

### Predict

支持3种输入格式, `http/ftp`, `oss`, `base64str`

输入样例:

```json
{
    "_k12.predict_images": [
        {
            "name": "1-35582.jpg",
            "content": "https://data-platform.s3.didiyunapi.com/datasets/user01/uuid01/mnist01/train/1/35582.jpg"
        },
        {
            "name": "6-49837.jpg",
            "content": "oss://data-platform/datasets/user01/uuid01/mnist01/train/6/49837.jpg"
        },
        {
            "name": "5-16451.jpg",
            "content": "/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/wAALCAAcABwBAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/9oACAEBAAA/APn+uw8OfDfWfEmk/wBqLdabp1kzmOGbUbjyRO46hODn09OvpWb4j8GeIPCk5j1jTJoE3YWcDdE+emHHB6dOtYNKoBYAnAzyfSvX/il4W8R6jrmn6doWg31zoWm2EUVjLawF45AQGZ8qMbiT06nGa4i+1Pxlofh+fw7qq6la6deFWNtfQMudrBgU3jI5A6Vy1Feq6R4N+KDafE91rF7oOlRIrCa+1NoEjQ8fdDZHToQO1R+OvGGnHwXB4QtNYvPEdwlwLifVbonajAY2RZ5I68nPU4znjy6ir+o67q+sCManqt9fCPhBc3Dy7fpuJxVCiv/Z"
        }
    ]
}
```

输出样例:

```
[
    {
         'version': '0.1.0',
         'server': 'localhost:8119',
         'type': 'metrics',
         'appId': 'k12ai.popkey',
         'token': '123456',
         'user': 'user01',
         'op': 'predict.start',
         'service_name': 'k12cv',
         'service_uuid': 'uuid01',
         'timestamp': 1626060328328,
         'datetime': '20210712112528',
         'data': [
             {
                   '_id_': 'd02280eaf0363a36',
                   'category': 'predict',
                   'type': 'text',
                   'data': {
                       'title': 'predict_target-0', 
                       'payload': '1-35582.jpg: 1'     // 1-35582.jpg 为输入时`name`属性, 1 为最终的输出标签
                   }
             }
         ]
    }
    ...
]
```

In [44]:
test_params = {
    '_k12.predict_images': [
        {
            'name': '1-35582.jpg',
            'content': 'https://data-platform.s3.didiyunapi.com/datasets/user01/uuid01/mnist01/train/1/35582.jpg'
        },
        {
            'name': '6-49837.jpg',
            'content': 'oss://data-platform/datasets/user01/uuid01/mnist01/train/6/49837.jpg'
        },
        {
            'name': '5-16451.jpg',
            'content': img2b64('https://data-platform.s3.didiyunapi.com/datasets/user01/uuid01/mnist01/train/5/16451.jpg')
        }
    ]
}

data = json.loads('''{
    "op":"predict.start",
    "token": "123456",
    "appId": "%s",
    "user": "%s",
    "service_name": "k12cv",
    "service_uuid": "%s",
    "service_params": %s
}''' % (appId, user, uuid, json.dumps(test_params)))

api = 'http://%s:%d/%s' % (host, port, uri)
print("Input:\n")
print(data)
json.loads(requests.post(url=api, json=data).text)

Input:

{'op': 'predict.start', 'token': '123456', 'appId': 'k12ai.popkey', 'user': 'user01', 'service_name': 'k12cv', 'service_uuid': 'uuid01', 'service_params': {'_k12.predict_images': [{'name': '1-35582.jpg', 'content': 'https://data-platform.s3.didiyunapi.com/datasets/user01/uuid01/mnist01/train/1/35582.jpg'}, {'name': '6-49837.jpg', 'content': 'oss://data-platform/datasets/user01/uuid01/mnist01/train/6/49837.jpg'}, {'name': '5-16451.jpg', 'content': '/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/wAALCAAcABwBAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/9oACAEBAAA/APn+uw8OfDfWfEmk/wBqLdabp1kzmOGbUbjyRO46hODn09OvpWb4j8GeIPCk5j1jTJoE3YWcDdE+emHHB6dOtYNKoBYAnAzyfSvX/il4

{'code': 100000, 'message': {'en': 'success', 'cn': '成功'}}

In [45]:
for _ in range(30):
    result = requests.get(url=f'{popapi}?key={appId}.metrics').text
    result = json.loads(result)
    if len(result) >= 3:
        print(result)
        break
    time.sleep(1)

['{"version": "0.1.0", "server": "localhost:8119", "type": "metrics", "appId": "k12ai.popkey", "token": "123456", "user": "user01", "op": "predict.start", "service_name": "k12cv", "service_uuid": "uuid01", "timestamp": 1626061328031, "datetime": "20210712114208", "data": [{"_id_": "d02280eaf0363a36", "category": "predict", "type": "text", "data": {"title": "predict_target-0", "payload": "1-35582.jpg: 1"}}]}', '{"version": "0.1.0", "server": "localhost:8119", "type": "metrics", "appId": "k12ai.popkey", "token": "123456", "user": "user01", "op": "predict.start", "service_name": "k12cv", "service_uuid": "uuid01", "timestamp": 1626061328112, "datetime": "20210712114208", "data": [{"_id_": "830ea7d91a60485d", "category": "predict", "type": "text", "data": {"title": "predict_target-1", "payload": "5-16451.jpg: 5"}}]}', '{"version": "0.1.0", "server": "localhost:8119", "type": "metrics", "appId": "k12ai.popkey", "token": "123456", "user": "user01", "op": "predict.start", "service_name": "k12c

## 压测 (P40 跑10个任务)

In [ ]:
# from k12libs.utils.nb_easy import k12ai_train_execute
# keys = k12ai_train_execute(
#     framework='k12cv', task='cls',
#     network=backbones[-1], dataset='rmnist',
#     batchsize=32, inputsize=32, epoch_num=20, run_num=10)